**This notebook is an exercise in the [Data Cleaning](https://www.kaggle.com/learn/data-cleaning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/inconsistent-data-entry).**

---


In this exercise, you'll apply what you learned in the **Inconsistent data entry** tutorial.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [76]:
from learntools.core import binder
binder.bind(globals())
from learntools.data_cleaning.ex5 import *
print("Setup Complete")

Setup Complete


# Get our environment set up

The first thing we'll need to do is load in the libraries and dataset we'll be using.  We use the same dataset from the tutorial.

In [77]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

# read in all our data
professors = pd.read_csv("../input/pakistan-intellectual-capital/pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

In [78]:
professors.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [79]:
professors.describe(include='all')

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
count,1142.000000,1142.000000,1142,1142,1142,1142,1123,1138,1142,1142,489.000000,623,124
unique,NaN,NaN,1133,63,17,5,46,42,350,38,NaN,571,52
top,NaN,NaN,Muhammad Zubair,Sir Syed University of Engineering and Technology,Computer Science,Punjab,Assistant Professor,MS,FAST– National University of Computer and Emer...,Pakistan,NaN,Software Engineering,PhD in progress
freq,NaN,NaN,3,81,744,449,489,463,67,776,NaN,16,27
mean,1054.350263,1055.350263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.456033,NaN,NaN
std,520.202783,520.202783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.577565,NaN,NaN
min,2.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1983.000000,NaN,NaN
25%,689.250000,690.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,NaN
50%,1087.500000,1088.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.000000,NaN,NaN
75%,1476.750000,1477.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.000000,NaN,NaN


In [80]:
professors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 1142 non-null   int64  
 1   S#                                         1142 non-null   int64  
 2   Teacher Name                               1142 non-null   object 
 3   University Currently Teaching              1142 non-null   object 
 4   Department                                 1142 non-null   object 
 5   Province University Located                1142 non-null   object 
 6   Designation                                1123 non-null   object 
 7   Terminal Degree                            1138 non-null   object 
 8   Graduated from                             1142 non-null   object 
 9   Country                                    1142 non-null   object 
 10  Year                    

Next, we'll redo all of the work that we did in the tutorial.

In [81]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

# get the top 10 closest matches to "south korea"
countries = professors['Country'].unique()
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")
    
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")
countries = professors['Country'].unique()

All done!


# 1) Examine another column

Write code below to take a look at all the unique values in the "Graduated from" column.

In [82]:
# TODO: Your code here
professors["Graduated from"].unique()

array(['Asian Institute of Technology',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'University of Balochistan',
       "Sardar Bahadur Khan Women's University",
       'SRH Hochschule Heidelberg',
       'Institute of Business Administration,Karachi', 'DUET,Karachi',
       'University of Turbat', 'University of Vienna',
       'Monash University', 'University of Stirling',
       'Chinese Academy of Sciences', 'University of Innsbruck',
       'Vienna University of Technology', 'University of Paris-Est',
       'The University of Cambridge', 'Harbin Institute of Technology',
       'University of Nice, Sophia Antipolis', 'The University of York',
       'Galilée - Université Paris 13', 'University of Bedfordshire',
       'North Dakota State University', 'Kyungpook National University',
       'The University of Manchester',
       'National University of Sciences and Technology',
       'FAST– National University of Computer

Do you notice any inconsistencies in the data?  Can any of the inconsistencies in the data be fixed by removing white spaces at the beginning and end of cells?

Once you have answered these questions, run the code cell below to get credit for your work.

In [83]:
# Check your answer (Run this code cell to receive credit!)
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

There are inconsistencies that can be fixed by removing white spaces at the beginning and end of cells.  For instance, "University of Central Florida" and " University of Central Florida" both appear in the column.

In [84]:
# Line below will give you a hint
q1.hint()
q1.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use `unis = professors['Graduated from'].unique()` to take a look at the unique values in the 'Graduated from' column. You may find it useful to sort the data before printing it.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> There are inconsistencies that can be fixed by removing white spaces at the beginning and end of cells.  For instance, "University of Central Florida" and " University of Central Florida" both appear in the column.

# 2) Do some text pre-processing

Convert every entry in the "Graduated from" column in the `professors` DataFrame to remove white spaces at the beginning and end of cells.

In [85]:
# TODO: Your code here
professors['Graduated from'] = professors['Graduated from'].str.strip()
print(professors['Graduated from'])

# Check your answer
q2.check()

0                           Asian Institute of Technology
1                           Asian Institute of Technology
2                           Asian Institute of Technology
3                           Asian Institute of Technology
4       Balochistan University of Information Technolo...
                              ...                        
1137                                                  JNU
1138                                             Grenoble
1139                          Florida Atlantic University
1140    Ghulam Ishaq Khan Institute of Science and Tec...
1141                            Seoul National University
Name: Graduated from, Length: 1142, dtype: object


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [86]:
# Lines below will give you a hint or solution code
q2.hint()
q2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> In the tutorial, you did the same operation on a different column.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
professors['Graduated from'] = professors['Graduated from'].str.strip()

```

# 3) Continue working with countries

In the tutorial, we focused on cleaning up inconsistencies in the "Country" column.  Run the code cell below to view the list of unique values that we ended with.

In [87]:
# get all the unique values in the 'City' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Take another look at the "Country" column and see if there's any more data cleaning we need to do.

It looks like 'usa' and 'usofa' should be the same country.  Correct the "Country" column in the dataframe to replace 'usofa' with 'usa'.

**Use the most recent version of the DataFrame (with the whitespaces at the beginning and end of cells removed) from question 2.**

In [88]:
# TODO: Your code here!
matches = fuzzywuzzy.process.extract("usa", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
replace_matches_in_column(df=professors, column='Country', string_to_match="usa", min_ratio=70)

# Check your answer
q3.check()

All done!


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [91]:
countries = professors['Country'].unique()
countries

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usa', 'south korea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'finland', 'netherland',
       'greece', 'turkey', 'macau', 'singapore', 'spain', 'japan',
       'hongkong', 'saudi arabia', 'mauritius', 'scotland'], dtype=object)

In [90]:
# Lines below will give you a hint or solution code
q3.hint()
q3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use the `replace_matches_in_column()` function defined above.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
matches = fuzzywuzzy.process.extract("usa", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
replace_matches_in_column(df=professors, column='Country', string_to_match="usa", min_ratio=70)

```

# Congratulations!

Congratulations for completing the **Data Cleaning** course on Kaggle Learn!

To practice your new skills, you're encouraged to download and investigate some of [Kaggle's Datasets](https://www.kaggle.com/datasets).

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*